In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import copy
import random
np.set_printoptions(suppress=True, threshold=np.inf)

In [5]:
## Import tensorflow package for modeling
import torch
from torch.autograd import Variable
import torch.optim as optim

## Data processing
import pandas as pd
import numpy as np

## Min-max normalization
from sklearn.preprocessing import MinMaxScaler

## Plot the graph
import matplotlib.pyplot as plt
%matplotlib inline

## Initializing module
from sklearn.linear_model import LinearRegression
np.set_printoptions(suppress=True)

## Copy module
import copy

## Used to calculate the training time
import time

## Set the GUP environment
import os
import sys

from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker

In [6]:
## Set up the display
torch.set_printoptions(sci_mode=False)

## Data preparation

### Label encoder

In [7]:
demand_x = pd.read_csv("demand_x.csv")
demand_y = pd.read_csv("demand_y.csv")

In [8]:
demand_y.insert(0, "Number", demand_x["Number"])

In [9]:
date = demand_x.iloc[:,0]
demand_x = demand_x.iloc[:,1:]

In [10]:
labelencoder = LabelEncoder()
demand_x_encode = copy.deepcopy(demand_x)
demand_y_encode = copy.deepcopy(demand_y)

In [11]:
demand_x_encode["Number"]=labelencoder.fit_transform(demand_x_encode["Number"])
demand_y_encode["Number"]=labelencoder.transform(demand_y_encode["Number"])

In [12]:
material_num = [i for i in range(20)]
print(labelencoder.inverse_transform(material_num))

['CC110105002300' 'CC110106001800' 'CC110106004900' 'CC110108002300'
 'CC110108002800' 'CC110108002850' 'CC110108003400' 'CC110108004000'
 'CC110108004300' 'CC110108004500' 'CC110108004700' 'CC110203504500'
 'CC110208001800' 'CC110208002500' 'CC110208003600' 'CC110210002800'
 'CC110215002700' 'CC110215003050' 'CC110215004750' 'CC110220004600']


### Data shuffle

In [17]:
zip_list = list(zip(demand_x_encode.values.tolist(), demand_y_encode.values.tolist()))
random.shuffle(zip_list)
demand_x_shuffle, demand_y_shuffle = zip(*zip_list)
demand_x_shuffle, demand_y_shuffle = np.array(demand_x_shuffle), np.array(demand_y_shuffle)

In [19]:
demand_y_shuffle

array([[  10.  ,    0.  ,  105.  ,  279.6 ,    0.  ],
       [   3.  ,    0.  ,    7.  ,   21.5 ,   10.  ],
       [  11.  ,  167.  ,   21.  ,    0.  ,    0.  ],
       [  11.  ,    0.  ,    0.  ,    0.  ,    0.  ],
       [   9.  ,  175.5 ,    0.  ,    0.  ,  228.  ],
       [   6.  ,    0.  ,    0.  ,    0.  ,    0.  ],
       [   6.  ,    0.  ,  147.5 ,   92.5 ,    0.  ],
       [   4.  ,   18.  ,    0.  ,    0.  ,   28.5 ],
       [   6.  ,    0.  ,    0.  ,    0.  ,    0.  ],
       [  18.  ,    0.  ,   41.  ,    0.  ,  136.08],
       [  19.  ,    0.  ,    0.  ,    0.  ,    0.  ],
       [  15.  ,    0.  ,  502.5 ,    0.  ,    0.  ],
       [   3.  ,   28.5 ,    0.  ,    0.  ,    5.  ],
       [   1.  ,    0.  ,    4.  ,    0.  ,  411.  ],
       [  10.  ,    0.  ,    0.  ,    0.  ,   67.  ],
       [  10.  ,    0.  ,    0.  ,    0.  ,   80.2 ],
       [  12.  ,   45.  ,    0.  ,    0.  ,    0.  ],
       [   3.  ,    7.  ,   21.5 ,   10.  ,   14.  ],
       [  15.  ,   19.63,   

In [306]:
x_sets_train, y_sets_train, x_sets_test, y_sets_test = [],[],[],[]

for _ in range(15):
    block_x_train, block_y_train, block_x_test, block_y_test = get_data(demand_x_encode, demand_y_encode, data_des,number_encode)

    zip_list = list(zip(block_x_train.tolist(), block_y_train.tolist()))
    random.shuffle(zip_list)
    demand_x_shuffle, demand_y_shuffle = zip(*zip_list)
    demand_x_shuffle, demand_y_shuffle = np.array(demand_x_shuffle), np.array(demand_y_shuffle)
    
    x_sets_train.append(demand_x_shuffle)
    y_sets_train.append(demand_y_shuffle)
    x_sets_test.append(block_x_test)
    y_sets_test.append(block_y_test)

In [308]:
x_sets_train[][:10]

array([[  3.  ,  11.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ],
       [  3.  ,   4.  ,  22.84,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         91.  ,  91.  ],
       [  0.  ,  11.  ,   0.1 ,   0.  ,   0.  ,   0.  ,  39.5 , 356.4 ,
        376.1 ,   0.  ],
       [ 13.  ,   7.  ,  14.5 ,  57.  ,   0.  , 146.  ,  15.  ,  11.5 ,
          7.  ,   0.  ],
       [ 10.  ,  11.  ,   0.  ,   0.  , 313.5 ,   0.  ,   0.  , 164.  ,
          0.  ,  20.  ],
       [  3.  ,   7.  ,  43.  ,  15.5 ,   0.  ,  46.  ,   0.  ,  21.5 ,
          0.  ,   0.  ],
       [  4.  ,   4.  ,  73.  ,   0.  ,   0.  ,   0.  , 360.  ,   0.  ,
          0.  ,   0.  ],
       [  1.  ,  12.  ,   0.  ,   1.5 ,   0.  ,   0.  ,   1.  ,   0.  ,
          0.  , 635.38],
       [  3.  ,   6.  ,   0.  ,   0.  ,   0.  ,   0.  ,  43.  ,  15.5 ,
          0.  ,  46.  ],
       [ 11.  ,   3.  , 363.2 ,   0.  ,   0.  ,   0.  ,   0.  , 981.1 ,
          0.  ,   0.  ]])